In [3]:
import requests

## Question 1

First, we define our request URL with its query parameters. Using the Requests library, we send a GET request with the given URL and query parameters. I chose option 2 (to filter by type as we make the HTTP request) because it reduces the amount of data we are requesting from mbta therefore reducing the amount of data we have to process on our end.

In [4]:
URL = "https://api-v3.mbta.com/routes"
types = "0,1"
query_params = {'filter[type]': types}

request = requests.get(url = URL, params = query_params)

data = request.json()
for i in data['data']:
    print(i['attributes']['long_name'])

Red Line
Mattapan Trolley
Orange Line
Green Line B
Green Line C
Green Line D
Green Line E
Blue Line


## Question 2

As I read this question, I began to think it may be better to represent routes and stops as objects instead of having to iterate through the response from question 1. In order to find how many stops are on a given route, we need to make a GET request to endpoint `https://api-v3.mbta.com/stops?filter[route]=TODO`  

In [5]:
class Route:
    def __init__(self, _id, long_name):
        self._id = _id
        self.long_name = long_name

In [6]:
routes = []
for i in data['data']:
    route = Route(i['id'], i['attributes']['long_name'])
    routes.append(route)

In [7]:
class Stop:
    def __init__(self, _id, name, route):
        self._id = _id
        self.name = name
        self.route = route

In [8]:
URL = "https://api-v3.mbta.com/stops"

# For parts 1 and 2
stops = []
routes_with_stops = {}

# For part 3, dict with key: stop, value: [# routes that stop here, [names of routes]]
stops_with_routes = {}
for route in routes:
    route_id = route._id

    query_params = {'filter[route]': ('%s'%route_id)}
    request = requests.get(url = URL, params = query_params)
    data = request.json()
    
    routes_with_stops[route_id] = 0
    
    for i in data['data']:
        stop = Stop(i['id'], i['attributes']['name'], route_id)
        stops.append(stop)
        routes_with_stops[route_id] += 1
        
        if stop.name in stops_with_routes.keys():
            stops_with_routes[stop.name][0] += 1
            if route_id not in stops_with_routes[stop.name][1]: 
                stops_with_routes[stop.name][1].append(route_id) 
        else:
            stops_with_routes[stop.name] = [1, [route_id]]

        
most_stops = float("-inf")
least_stops = float("inf")
route_most_stops = ""
route_least_stops = ""
for route,stop in routes_with_stops.items():
    if stop > most_stops:
        most_stops = stop
        route_most_stops = route
    if stop < least_stops:
        least_stops = stop
        route_least_stops = route

### Part 1

In [9]:
print("Subway route with most stops: %s (%s stops)"%(route_most_stops, most_stops))

Subway route with most stops: Green-B (24 stops)


##